In [1]:
#爬虫爬取过去一年猪肉价格

#0.创建数据表
import pymysql

def create_table():
    if not connect_db() == False:
        conn, cur = connect_db()
        sql = """create table copredict_source_pork(
                 id int(10) auto_increment primary key,
                 date date,
                 region varchar(64),
                 type varchar(64),
                 price float)"""
        cur.execute(sql)
    
        cur.execute('desc copredict_source_pork')
        print(cur.fetchall())

#1.数据爬取部分-----------------------------------------------------------------------------------

import pandas as pd   #载入pandas模块
import requests       #载入request模块
import time           #载入time模块

#生成日期列表
def dataRange(a,b):    #定义获取日期函数
    fmt = '%Y-%m-%d'   #定义返回的日期格式
    bgn = int(time.mktime(time.strptime(a,fmt)))  #定义开始日期
    end = int(time.mktime(time.strptime(b,fmt)))  #定义结束日期
    list_date = [time.strftime(fmt,time.localtime(i))for i in range(bgn,end+1,3600*24)]  #赋值日期表的值，值为日期
    return list_date   #返回每一天日期列表


    #获得网站回应
def get_json(url):                                      #定义获取数据的函数，以json文件返回
    headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36'
    }                                                  #赋值headers，内容从分析网页的XHR找，以供下面的代码使用
    try:                                               #建立try-except，获取数据时防止出错
        response = requests.get(url, headers=headers)  #获取数据
        if response.status_code == 200:               #如果获取数据请求有回应（成功）
            json_text = response.json()               #就用文本返回数据，用json格式保存
            return json_text                         #返回此文本格式
    except Exception:                                #如果上述情况并未发生（请求失败）
        print('An Error has occured on this page!')  #显示“An Error has occured on this page!”
        return None                                 #么得返回
    
    
    #获取网页数据
def get_contents(url):                        #定义爬虫主函数
    doc = get_json(url)                       #使用上述定义的函数获取网址
    
    dic = {}                                  #定义一个用于储存爬取数据的空字典
    dic['pigprice']   =  doc['pigprice']      #获取pigprice数据
    #dic['pig_in']     =  doc['pig_in']       #获取pig_in数据
    #dic['pig_local']  =  doc['pig_local']    #获取pig_local数据
    #dic['maizeprice'] =  doc['maizeprice']   #获取maizeprice数据
    #dic['bean']       =  doc['bean']         #获取bean数据
  
    a = '-'.join(doc['time'][3])                              #取最早的时间
    b = time.strftime('%Y-%m-%d',time.localtime(time.time())) #取当前时间
    
    dic['date'] = dataRange(a,b)                              #取得每一天日期列表
    
    return pd.DataFrame(dic)                                 #返回带有日期与数据的表格，用pandas格式


#2.连接数据库-----------------------------------------------------------------------------------    

import pymysql

def connect_db():
    try:
        #建立connect路径
        conn = pymysql.connect(host = 'sql.b51.vhostgo.com', db = 'passvisor', user = 'passvisor', password = 'Xudamin123', charset = 'utf8')
        #建立cursor
        cur = conn.cursor()  
        return conn, cur    #返回路径、cursor，供下文使用
    except:
        return False       #如上述代码请求不成功，返回False字样



#3.显示数据-----------------------------------------------------------------------------------    

import pymysql

def show_table_data(table_name):                #定义显示数据表函数
    if not connect_db == False:                #如果连接数据库没有不成功，
        sql = 'select * from %s'%(table_name)   #sql指令执行全选数据表内容
        conn, cur = connect_db()                #connect路径与cursor来自connect_db()
        cur.execute(sql)                        #执行sql指令
        print(cur.fetchall())                   #显示所有

#4.读取数据表-----------------------------------------------------------------------------------

import pandas as pd
import pymysql

def read_table_data(table_name):
    if not connect_db() == False:                     #如果连接数据库没有不成功，
        conn, cur = connect_db()                       #connect路径与cursor来自connect_db()
        sql = 'select * from %s'%(table_name)         #sql指令执行全选数据表内容
        db_data = pd.read_sql(sql = sql, con = conn) #用pandas格式读取
    return db_data                                  #显示数据

#print(readdata())



#5.更新数据并写入数据库-----------------------------------------------------------------------------------

import pymysql
import datetime

def updated_data():                       
    if not connect_db == False:         #如果连接数据库没有不成功，
        conn, cur = connect_db()         #connect路径与cursor来自connect_db() 
        
        s_data = get_contents('https://zhujia.zhuwang.cc/api/chartData?areaId=-1&aa=1581735399831')  #定义s_data为最新爬取的数据
        db_data = read_table_data('copredict_source_pork')                                           #定义db_data已有数据
        k = 0                                                                                        #赋值k
        
        for i in range(len(s_data)):                                                                #建立for循环，开始更新数据  
            #如果最新爬取的数据记录里的日期没有存在于已有数据内时，（datetime代码为将str转换成datetime.date格式）
            if datetime.datetime.strptime(s_data['date'][i], '%Y-%m-%d').date() not in list(db_data['date']):  
                #写入该记录
                sql = "insert into copredict_source_pork(date, region, type, price) value('%s', '%s', '%s', %s)"%(s_data['date'][i], 'GD', 'WSY', s_data['pigprice'][i])
                cur.execute(sql)  #执行sql指令
                k = k + 1         #为k赋值，若有变化就+1
        conn.commit()
        print('Done! Update {} records'.format(k))   #显示结果

    
#6.运行程序-----------------------------------------------------------------------------------

if __name__ == '__main__': 
    updated_data()              #从入口程序运行程序
   
    #db_data = read_table_data('copredict_source_pork')
    
    #show_table_data('copredict_source_pork')

    #get_contents('https://zhujia.zhuwang.cc/api/chartData?areaId=-1&aa=1581735399831')
    
    #create_table()

Done! Update 1 records
